In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix

In [29]:
import matplotlib.pyplot as plt
from matplotlib import style
style.use("ggplot")

Linear SVC (Support Vector Classifier) is to fit to the data you provide, returning a "best fit" hyperplane that divides, or categorizes, your data.

In [2]:
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load("en_core_web_sm")
stopwords = list(STOP_WORDS)

In [3]:
yelp_Data = pd.read_csv("yelp_labelled.txt",sep='\t',header=None)
yelp_Data.head(6)

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
5,Now I am getting angry and I want my damn pho.,0


In [4]:
#add column name
#here sentiment is postive or negative i.e 1 and 0 respectively.
column = ["Review","Sentiment"]
yelp_Data.columns=column
yelp_Data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
yelp_Data.shape

(1000, 2)

In [6]:
# now need to load amazon data 
Amazon_data = pd.read_csv("amazon_cells_labelled.txt",sep='\t',header=None)
Amazon_data.columns = column
Amazon_data.head()


,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [7]:
Amazon_data.shape

(1000, 2)

In [8]:
#now need to load imdb data
Imdb_data = pd.read_csv("imdb_labelled.txt",sep='\t',header=None)
Imdb_data.columns = column
Imdb_data.head()


,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [9]:
Imdb_data.shape

(748, 2)

In [10]:
merge_data = yelp_Data.append([Amazon_data,Imdb_data],ignore_index=True)
merge_data.shape

(2748, 2)

In [11]:
merge_data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [12]:
merge_data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [13]:
merge_data.isnull().sum()


Review       0
Sentiment    0
dtype: int64

In [14]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [45]:
#In the function text_data_cleaning() we are first converting the sentence into tokens. 
#Then for each token we are performing lemmatization. 
#If the lemma is not a pronoun we are converting the lemma to lower case; 
#else we are converting the token to lower case. 
#Finally we are removing all the stopwords and punctuation mark
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_!="-PRON-":              # here doing lemmatization 
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    
    for token in tokens:
        if token not in stopwords and token not in punct:              # here removing the stopwords and punctions
            cleaned_tokens.append(token)
    return cleaned_tokens


text_data_cleaning("heloo)) ##1!! world - rahul is not coming ")

['heloo', '1', 'world', 'rahul', 'come']

In [60]:
# Vectorizatiob Feature Engineering (TF-IDF)
tfidf_vect = TfidfVectorizer(tokenizer= text_data_cleaning,ngram_range=(2, 2),norm ='l1')
classifier = LinearSVC()

In [61]:
X = merge_data['Review']
y = merge_data["Sentiment"]

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((2198,), (550,))

In [63]:
clf = Pipeline([('tfidf', tfidf_vect), ('clf', classifier)])
clf.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l1', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [64]:
y_pred = clf.predict(X_test)

In [65]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.80      0.78       285
           1       0.77      0.72      0.75       265

   micro avg       0.77      0.77      0.77       550
   macro avg       0.77      0.76      0.76       550
weighted avg       0.77      0.77      0.76       550



In [66]:
confusion_matrix(y_test, y_pred)

array([[229,  56],
       [ 73, 192]], dtype=int64)

In [67]:
accuracy_score(y_test, y_pred)

0.7654545454545455

In [39]:
clf.predict(['Wow, this is amazing lesson'])

array([1], dtype=int64)

In [40]:
clf.predict(['bad very bad  this is not amazing lesson'])

array([0], dtype=int64)

In [41]:
clf.predict(['Wow, this sucks'])

array([0], dtype=int64)

In [42]:
clf.predict(['Worth of watching it. Please like it'])

array([1], dtype=int64)

In [44]:
clf.predict(['Amazing'])

array([1], dtype=int64)